THOMAS MCDONNELL QUANT TRADING STRATEGIES HOMEWORK 3

In [332]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy as sp
import quandl
import functools
import seaborn as sns
import plotnine as p9
from datetime import datetime
from datetime import timedelta
from tqdm import tqdm
from pandas import DateOffset

import warnings

# Suppress the warning
warnings.filterwarnings("ignore")

Worked with Sean Lin

In [333]:
FR = pd.read_csv("C:/Users/thoma/Downloads/ZACKS_FR_2_f40c6a304f87d9f492c1f21839d474e2\ZACKS_FR_2_f40c6a304f87d9f492c1f21839d474e2.csv")

In [334]:
FR['per_end_date'] = pd.to_datetime(FR['per_end_date'])
start_date = datetime(2014, 6, 1)
FR = FR[FR['per_end_date'] >= start_date]
end_date = datetime(2022, 1, 31)
FR = FR[FR['per_end_date'] <= end_date]
FR = FR.set_index('ticker')

#Drop all of the tickers that don't have quarterly data
FR = FR[FR['per_type'] == 'Q']

In [335]:
FC = pd.read_csv("C:/Users/thoma\Downloads\ZACKS_FC_2_76e4bece47ce87cb8f221f639c7f829b/ZACKS_FC_2_76e4bece47ce87cb8f221f639c7f829b.csv")


In [336]:
FC = FC[['ticker','zacks_sector_code', 'per_end_date', 'filing_date']]

In [337]:
FC = FC.drop_duplicates(subset = 'ticker')

#Drop all tickers in the finance, industry and auto sector

FC = FC[~FC['zacks_sector_code'].isin([5.0, 13.0])]
FC = FC.set_index('ticker')

In [338]:
common_tickers = set(FC.index).intersection(set(FR.index))
FR = FR.loc[common_tickers]

In [339]:
FR['tot_debt_tot_equity'] = pd.to_numeric(FR['tot_debt_tot_equity'])

In [340]:
grouped = FR.groupby('ticker')

# Count the number of rows where tot_debt_tot_equity is greater than 0.1
counts = grouped['tot_debt_tot_equity'].apply(lambda x: (x > 0.1).sum())

# Filter out the tickers that don't have more than 85% of their rows with tot_debt_tot_equity greater than 0.1
FR = grouped.filter(lambda x: (x['tot_debt_tot_equity'] > 0.1).sum() > len(x)*.85)


In [341]:
FR = FR.reset_index()

In [342]:
FC = pd.read_csv("C:/Users/thoma\Downloads\ZACKS_FC_2_76e4bece47ce87cb8f221f639c7f829b/ZACKS_FC_2_76e4bece47ce87cb8f221f639c7f829b.csv")
FC = FC[['ticker', 'per_type','per_end_date', 'filing_date', 'tot_lterm_debt', 'net_lterm_debt', 'eps_diluted_net']]

In [343]:
FC = FC[FC['per_type'] == 'Q']

In [344]:
FC['per_end_date'] = pd.to_datetime(FC['per_end_date'])
FC['filing_date'] = pd.to_datetime(FC['filing_date'])
merged_df = pd.merge(FR, FC, on=['per_end_date', 'ticker'])
merged_df = merged_df.drop_duplicates()

In [345]:
merged_df = merged_df.set_index('ticker')

In [346]:
merged_df = merged_df.dropna(subset=['filing_date'])

In [347]:
unique_tickers = merged_df.index.unique()
unique_tickers

Index(['SRCL', 'EW', 'FLEX', 'MERC', 'BPL', 'FBIO', 'IEX', 'MLAB', 'EXAM',
       'CNL',
       ...
       'AMPH', 'GWR', 'ADP', 'TCPTF', 'CMCO', 'PTSI', 'TXN', 'SUNEQ', 'SXC',
       'NTI'],
      dtype='object', name='ticker', length=1892)

In [348]:
ticker_counts = {}
for ticker in tqdm(unique_tickers):
    # Get the number of rows for the current ticker
    num_rows = merged_df.loc[merged_df.index == ticker].shape[0]
    # Add the ticker and number of rows to the dictionary
    ticker_counts[ticker] = num_rows


100%|██████████| 1892/1892 [00:02<00:00, 817.02it/s]


In [349]:
#drop all tickers where they dont have at least 28 instances of data
tickers_to_drop = [ticker for ticker in ticker_counts if ticker_counts[ticker] != 31]

# Drop rows where the 'ticker' column is in the list of tickers to drop
merged_df = merged_df[~merged_df.index.isin(tickers_to_drop)]
FR
unique_tickers = merged_df.index.unique().tolist()

In [350]:
tickers_to_remove = ['SWIOU', 'GEF.B' 'SENEB', 'BUKS', 'NUVR', 'INRD', 'TMS.', 'GBCS', 'UNIR', 'JANL', 'STCC', 'SMID', 'TPCS', 'TDSNA', 'BVHBB', 'BGSF']

In [351]:
tickers = [x for x in unique_tickers if x not in tickers_to_remove]

Now I have a list of tickers whose data I will be taking to calculate the financial ratios. All of the work below here is about pulling in ZACKs dataframes and extracting all of the data. 

For all ZACK tables that aren't FC I will also need to merge my data on the filing date again to ensure that I have a filing date

In [352]:
merged_df = merged_df.loc[tickers]

In [353]:
merged_df = merged_df[['per_end_date', 'filing_date', 'tot_debt_tot_equity', 'ret_invst', 'net_lterm_debt', 'tot_lterm_debt', 'eps_diluted_net']]

In [354]:
filtered_df = merged_df[merged_df.index.isin(tickers)]

to do tomorrow: add SHRS/shares_out, MKTV/MKT_VAL, deal with getting prices on per end dates, shift all of my data to us knowing it the day after the filing date instead of on the filing date, change all EPS to equal .001 if negative, if net_lterm_debt is all NA, drop the ticker, if only a few NA set to 0. 

In [355]:
filtered_df

,per_end_date,filing_date,tot_debt_tot_equity,ret_invst,net_lterm_debt,tot_lterm_debt,eps_diluted_net
ticker,,,,,,,
SRCL,2014-06-30,2014-08-07,0.9260,2.4201,182.6799,1563.913,0.95
SRCL,2014-09-30,2014-11-07,0.8921,2.4478,141.5759,1512.333,0.96
SRCL,2014-12-31,2015-03-02,0.8654,2.3998,101.7399,1527.246,0.97
SRCL,2015-03-31,2015-05-07,0.8079,0.8585,-70.3260,1462.177,0.34
SRCL,2015-06-30,2015-08-05,0.8416,2.5073,-10.3911,1538.736,1.02
...,...,...,...,...,...,...,...
SXC,2020-12-31,2021-02-25,1.3514,-0.2979,-48.9000,673.900,-0.07
SXC,2021-03-31,2021-04-28,1.2563,1.5762,-0.7000,641.600,0.20
SXC,2021-06-30,2021-07-29,1.2980,-0.6502,-110.7000,650.200,-0.11


Dealing with Shares

In [356]:
SHRS = pd.read_csv("C:/Users/thoma/Downloads/ZACKS_SHRS_2_99db6fa97ac677f3c0d45a9fa9a70196\ZACKS_SHRS_2_99db6fa97ac677f3c0d45a9fa9a70196.csv")
SHRS = SHRS[['ticker', 'per_end_date', 'shares_out']]

In [357]:
SHRS = SHRS.set_index('ticker')

Dealing with market value

In [358]:
MKTV = pd.read_csv("C:/Users/thoma/Downloads/ZACKS_MKTV_2_ecb7f768974bbdd26964caefe2fd0378\ZACKS_MKTV_2_ecb7f768974bbdd26964caefe2fd0378.csv")
MKTV = MKTV[['ticker', 'per_end_date', 'mkt_val']]

In [359]:
MKTV['per_end_date'] = pd.to_datetime(MKTV['per_end_date'])

In [360]:
filtered_df = pd.merge(filtered_df, MKTV, left_on = ['ticker', 'per_end_date'], right_on = ['ticker', 'per_end_date'])

Turning EPS all positive

In [361]:
filtered_df.loc[:, 'eps_diluted_net'] = filtered_df.loc[:, 'eps_diluted_net'].where(filtered_df.loc[:, 'eps_diluted_net'] >= 0, 0.001)

In [362]:
filtered_df = filtered_df.set_index('ticker')

In [363]:
filtered_df['filing_date'] = filtered_df['filing_date'] + DateOffset(days=1)

import price data

In [364]:
eod_price = quandl.get_table('QUOTEMEDIA/PRICES', ticker = ['AAPL'], api_key = 'uoxc6V3s61czWuHoGLcs',
                    qopts = { 'columns': ['ticker', 'date', 'adj_close'] }, 
                    date = { 'gte': '2014-08-01', 'lte': '2022-02-01' }, 
                     paginate=True).set_index('date')[['adj_close']]
for ticker in tqdm(unique_tickers):
    df =  quandl.get_table('QUOTEMEDIA/PRICES', ticker = [ticker], api_key = 'uoxc6V3s61czWuHoGLcs',
                    qopts = { 'columns': ['ticker', 'date', 'adj_close'] }, 
                    date = { 'gte': '2014-08-01', 'lte': '2022-02-01' }, 
                     paginate=True)
    eod_price = eod_price.append(df)


100%|██████████| 1012/1012 [11:03<00:00,  1.53it/s]


In [365]:
eod_price = eod_price[['ticker', 'date', 'adj_close']]

In [366]:
eod_price.dropna(axis=0, how='any', subset=['ticker'], inplace=True)

In [367]:
eod_price = eod_price.sort_values(by='date')

In [368]:
filtered_df = filtered_df.sort_values(by = 'filing_date')
filtered_df['date'] = filtered_df['filing_date']

In [369]:
n =pd.merge_asof(eod_price, filtered_df, by = 'ticker', left_on = 'date', right_on = 'date')

In [370]:
n = n.set_index('ticker')

In [371]:
LLY = n.loc['LLY']

In [372]:
n = n.reset_index()

In [373]:
tickers = n['ticker'].unique().tolist()

In [374]:
n = n.set_index('ticker')

In [375]:
n['net_lterm_debt'] = n['net_lterm_debt'].fillna(n['tot_lterm_debt'])
n['net_lterm_debt'] = n['net_lterm_debt'].fillna(0)

Loop to get my ratios

In [379]:
LLY = n.loc['LLY']

In [400]:
f = pd.DataFrame()
for ticker in tqdm(tickers):
    df = n.loc[ticker]
    df['date'] = pd.to_datetime(df['date'])
    df['per_end_date'] = pd.to_datetime(df['per_end_date'])
    df = df.merge(df, left_on='per_end_date', right_on='date')
    df = df.rename(columns = {'adj_close_y': 'per_end_price'})
    df['debt_to_market_cap'] = df['tot_debt_tot_equity_x']*df['per_end_price']/df['adj_close_x']
    df['price_to_earnings'] = df['adj_close_x']/df['eps_diluted_net_x']
    df['R'] = (df['ret_invst_x']*(df['net_lterm_debt_y'] + (df['mkt_val_x'])))
    df['ROI'] = df['R']/(df['net_lterm_debt_y']+df['mkt_val_x']*df['adj_close_x']/df['per_end_price']) 
    df['ticker'] = ticker
    f = f.append(df)
    f = f[['date_x', 'adj_close_x', 'debt_to_market_cap', 'price_to_earnings', 'ROI']]
    f = f.rename(columns = {'date_x': 'date', 'adj_close_x': 'price'})

100%|██████████| 972/972 [01:36<00:00, 10.04it/s]


In [411]:
f.to_csv('hw_3_ratio_data.csv')